In [123]:
import pandas as pd
import requests
import sqlite3
import re
from bs4 import BeautifulSoup

In [111]:
pt_london_2025_invited = "https://fabtcg.com/en/coverage/pro-tour-london/pairings/1/"
pt_london_2025 = "https://fabtcg.com/en/coverage/pro-tour-london/decklist/{}/"

In [223]:
def get_player_list(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    pairing = soup.find_all("div", {"class":"tournament-coverage__player-hero-and-deck"})
    decklist_urls = [re.search(r"\/decklist\/(\d+)", url) for url in (str(url) for url in (player.find("a") for player in pairing))]

    output = [url.group(1) for url in decklist_urls if url is not None]

    return output


def get_decklist(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html")

    decklist = [data.text.strip() for data in soup.find_all("td")]

    return decklist

In [ ]:
player_list = get_player_list(pt_london_2025_invited)
tournament_lists = [get_decklist(pt_london_2025.format(player_id)) for player_id in player_list]

In [240]:
tournament_lists_df = pd.DataFrame.from_records(tournament_lists)

long_list = pd.melt(tournament_lists_df, id_vars=[0,1,2,3,4], value_name="cards")
long_list[["copies","card"]] = long_list["cards"].str.split(" x ", expand=True)

long_list = long_list.drop([1, 2, 3, "variable", "cards"], axis=1).dropna()

long_list.to_clipboard(index=False)